In [2]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize

%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Number 1

In [ ]:
#load in via pandas
wbp = pd.read_json('world_bank_projects.json', orient = 'table')
wbpl = pd.read_json('world_bank_projects_less.json', orient = 'table')


In [ ]:
#bring together the two Df's
wbp2 = wbp.append(wbpl)
wbp2[['mjtheme','id']]

data = json.loads(wbp2)
json_normalize(wbp2['mjtheme'])

#I used .info(), .describe(), and .head() but removed them for brevity/polish

In [ ]:
#groupby country code, then find the size of each amalgamation, then sort
wbp2.groupby('countrycode').size().sort_values(ascending = False).head(10)

# Number 2

In [12]:
#after near infinite hours, decided to bring in using json and normalize
with open('world_bank_projects.json') as j:
    f1 = json.load(j)
df1 = json_normalize(f1, 'mjtheme_namecode',['id'])


with open('world_bank_projects_less.json') as j:
    f2 = json.load(j)
#normalize using the column we need to measure (ID as well so we can merge on it later)
df2 = json_normalize(f2, 'mjtheme_namecode',['id'])
#concatenate them together
df = df1.append(df2)
df.head()

,code,name,id
0,8,Human development,P129828
1,11,,P129828
2,1,Economic management,P144674
3,6,Social protection and risk management,P144674
4,5,Trade and integration,P145310


In [13]:
#groupby code and sort to find which codes are most used
df.groupby('code').size().sort_values(ascending = False)

code
11    251
10    216
8     211
2     199
6     169
4     146
7     130
5      77
9      50
1      39
3      15
dtype: int64

# Number 3


My plan was to bring in the names from the mjtheme column and join on ID with the mjtheme_namecode-containing table, but it is not working. I tried both bringing the json in and normalizing, and pd.read_json and splitting the lists that are contained within the various cells. I spent a good 6 hours trying various approaches, only to met with various errors that I could not figure out. Stackoverflow and I got really close, as did the forums on springboard. One of the mentors posted similar code to that shown below as a help for people in a similar situation to me.

In [19]:
#Use pandas to bring in a messy version with all the information
df0 = pd.read_json('world_bank_projects.json')
df1 = pd.read_json('world_bank_projects_less.json')

#bring in a JSON that can be refined to have the information we are particularly interested in
with open('world_bank_projects.json') as f:    
    raw = json.load(f)
with open('world_bank_projects_less.json') as f:    
    raw1 = json.load(f)

#We know ID will help us join the correct names later on
df_themes0 = json_normalize(raw, 'mjtheme_namecode', ['id'])
df_themes1 = json_normalize(raw1, 'mjtheme_namecode', ['id'])

df_themes = df_themes0.append(df_themes1)
df = df0.append(df1)

# Here we see a number of projects with multiple codes, meaning the project spans many themes
df_themes.groupby('id').code.nunique().sort_values(ascending=False).head(10)

# Addtionally we have the problems of a missing theme name. I was able to get this far, but then I was quickly derailed
print 'Top 10 Major World Bank Project Themes:'
print (df_themes.name.value_counts().head(10))

# The holy grail (at least in my tired opinion)
df_themes_name_to_code = df_themes.groupby('name').code.max().sort_values(ascending=False)

# drop the missing name rows
df_themes_name_to_code = df_themes_name_to_code[df_themes_name_to_code.index != '']

# convert to df
df_themes_name_to_code = pd.DataFrame(df_themes_name_to_code,columns=['code'])

# pull name into a column
df_themes_name_to_code['name_clean'] = df_themes_name_to_code.index

# set code to be the index
df_themes_code_to_name = df_themes_name_to_code.set_index(['code'])


# now merge on the name based on the code for the missing projects

df_themes_clean = df_themes.merge(df_themes_code_to_name,how='outer',left_on=['code'],right_index=True)

df_final = df_themes_clean.drop('name')

#drop is no cooperating?
print(df_final)



Top 10 Major World Bank Project Themes:
Environment and natural resources management    223
Rural development                               202
Human development                               198
Public sector governance                        184
Social protection and risk management           159
Financial and private sector development        130
                                                123
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Name: name, dtype: int64
     code               name       id         name_clean
0       8  Human development  P129828  Human development
17      8                     P127033  Human development
41      8  Human development  P132616  Human development
43      8  Human development  P146271  Human development
44      8  Human development  P146271  Human development
47      8  Human development  P146271  Human development
57      8  Human